In [1]:
""" Co-occurrence methods needed to obtain the co-occurrence representation
"""
import math
import numpy as np
import torch


from argparse import ArgumentParser
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.model_zoo as model_zoo
import torchvision.models as models
from PIL import Image


def ini_cooc_filter(channels, cooc_r):
    """ Method to obtain a co-occurrence filter
    given the number of channels and r
    """
    cooc_w = cooc_r * 2 + 1
    cooc_filter = np.ones((channels, channels, 1, 1))/math.pow(cooc_w, 2)
    for i in range(channels):
        cooc_filter[i, i, :, :] = 1e-10
    cooc_filter = torch.FloatTensor(cooc_filter).cuda()
    cooc_filter = cooc_filter.repeat(1, 1, cooc_w, cooc_w)
    return cooc_filter

def calc_spatial_cooc(tensor, cooc_filter, cooc_r):
    """ Method to obtain the co-occurrence representation
    given an activation tensor, co-occurrence filter and the size r
    """
    act_m = tensor > torch.mean(tensor)
    act = tensor * act_m.float()
    cooc_map = torch.nn.functional.conv2d(act, cooc_filter, padding=cooc_r)

    cooc_map = cooc_map / (tensor.shape[1] - 1)
    cooc_map = cooc_map * act_m.float()
    return cooc_map

Arguments

* tensor: activation 4D tensor with shape: (batch_size, rows, cols, channels). //En teoria es (batch, channels, rows, cols)

* cooc_filter: activation 4D tensor with shape: (channels, channels, 2 * cooc_r + 1, 2 * cooc_r + 1). This cooc_filter can be initialized using the function called ini_cooc_filter(channels, cooc_r) 
also in cooccurrences.py.

* cooc_r: is the radious considered for the co-occurrence window

Output

* cooc_tensor: activation 4D tensor with same shape than the input activation 4D tensor: (batch_size, rows, cols, channels).

In [ ]:
channels = 100
cooc_r = 2

cooc_w = cooc_r * 2 + 1
cooc_filter = np.ones((channels, channels, 1, 1))#/math.pow(cooc_w, 2)
for i in range(channels):
    cooc_filter[i, i, :, :] = 1e-10
cooc_filter = torch.FloatTensor(cooc_filter).cuda()
cooc_filter = cooc_filter.repeat(1, 1, cooc_w, cooc_w)

In [20]:
cooc_mapa = torch.rand((1,5,10,10), requires_grad=True).cuda()

cooc_mapb = torch.rand((1,5,15,15), requires_grad= True).cuda()


cooc_map1 = torch.mean(cooc_mapa, dim = [0,1])

cooc_map1 = torch.mean(cooc_mapa, dim = [0,1])

cooc_map1.size()

torch.Size([10, 10])

In [26]:
bbb =  torch.sum(torch.nn.functional.normalize(cooc_mapa, eps = 0, dim = [0,1], p=0))
print(bbb)

tensor(49.5607, device='cuda:0', grad_fn=<SumBackward0>)


In [27]:
aa = torch.sum(torch.div(cooc_mapa, torch.sum(cooc_mapa)))
print(aa)

tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)


In [ ]:
radius = 3

t = 0.8 #threshold o umbral
b,c,h,w = cooc_mapa.size()

aux = torch.zeros(cooc_mapa.size(), dtype=torch.bool)

for (b1,c1,h1,w1) in range((b,c,h,w)):
    for b2,c2,h2,w2 in cooc_mapa.size():
        if abs(h1-h2) > radius and abs(w1-w2) > radius and cooc_mapa[b2,c2,h2,w2] > t and cooc_mapa[b1,c1,h1,w1] > t:
            aux[b1,c1,h1,w1] = 1
        else:
            aux[b1,c1,h1,w1] = 0

In [ ]:
radius = 3

t = 0.1 #threshold o umbral
b,c,h,w = cooc_mapa.size()

aux = torch.zeros(cooc_mapa.size(), dtype=torch.bool)

for b1 in range(b):
    for c1 in range(c):
        for h1 in range(h):
            for w1 in range(w):
                for b2 in range(b):
                    for c2 in range(c):
                        for h2 in range(h):
                            for w2 in range(w):
                                if abs(h1-h2) > radius and abs(w1-w2) > radius and cooc_mapa[b2,c2,h2,w2] > t and cooc_mapa[b1,c1,h1,w1] > t:
                                    aux[b1,c1,h1,w1] = 1
                                else:
                                    aux[b1,c1,h1,w1] = 0

In [ ]:
aux

In [ ]:
#print(cooc_mapa > 0.99) #torch.mean(cooc_mapa))

aux = cooc_mapa > 0.2

cooc_mapc = cooc_mapa*aux

print(cooc_mapc)

In [ ]:
input = cooc_mapa

B, C, H, W = input.size()
x_flat = input.view(C, H * W)
grama = torch.mm(x_flat, x_flat.t())

In [ ]:
input = cooc_mapb

B, C, H, W = input.size()
x_flat = input.view(C, H * W)
gramb = torch.mm(x_flat, x_flat.t())

In [ ]:
grama.size()

In [ ]:
cooc_filter

In [ ]:
loss = torch.nn.MSELoss()
out = loss(grama, gramb)
#out.backward()

In [ ]:
out.size()

In [ ]:
cooc_filter = ini_cooc_filter(5, 1)
cooc_map_fixa = calc_spatial_cooc(cooc_mapa, cooc_filter, 1)

cooc_map_fixb = calc_spatial_cooc(cooc_mapb, cooc_filter, 1)

In [ ]:

loss(cooc_map_fixa, cooc_map_fixb)

In [ ]:



COOC_FILTER_PATH = "./data/weights_cooc_44_best_model_8192_ft.npy"

# Using VGG torch features from https://github.com/filipradenovic/cnnimageretrieval-pytorch
VGG16 = "http://cmp.felk.cvut.cz/cnnimageretrieval/data/networks/imagenet/imagenet-caffe-vgg16-features-d369c8e.pth"

def plot_figures(img, act_m, cooc_m_f, cooc_m_l):
    """ Method to plot the example figures
    """

    _, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15, 5))

    ax1.imshow(img)
    ax1.set(title='Image')
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax1.get_yticklabels(), visible=False)
    ax1.tick_params(axis='both', which='both', length=0)

    ax2.imshow(np.sum(act_m, axis=0))
    ax2.set(title='Activations')
    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.setp(ax2.get_yticklabels(), visible=False)
    ax2.tick_params(axis='both', which='both', length=0)

    ax3.imshow(np.sum(cooc_m_f, axis=0))
    ax3.set(title='Direct co-occurrences')
    plt.setp(ax3.get_xticklabels(), visible=False)
    plt.setp(ax3.get_yticklabels(), visible=False)
    ax3.tick_params(axis='both', which='both', length=0)

    ax4.imshow(np.sum(cooc_m_l, axis=0))
    ax4.set(title='Learned co-occurrences')
    plt.setp(ax4.get_xticklabels(), visible=False)
    plt.setp(ax4.get_yticklabels(), visible=False)
    ax4.tick_params(axis='both', which='both', length=0)

    plt.show()


if __name__ == '__main__':
    """ Main """

    parser = ArgumentParser()
    parser.add_argument('--test_image', dest='test_image', type=str,
                        default="./data/all_souls_000117.jpg", help='path to test image')
    parser.add_argument('--cooc_r', dest='cooc_r', type=int, default=4, help='cooc r size')

    args = parser.parse_args()

    # Load image and activation map
    image = Image.open(args.test_image)
    image = np.array(image)
    net_image = np.moveaxis(np.array(image), 2, 0)

    VGG16_net = models.vgg16(pretrained=False).features

    model_dir = os.path.join(os.path.join(os.getcwd(), 'data'), 'networks')
    VGG16_net.load_state_dict(model_zoo.load_url(VGG16, model_dir=model_dir))
    VGG16_net.cuda()
    VGG16_net.eval()

    net_input = torch.FloatTensor(np.expand_dims(net_image, axis=0)).cuda()
    act_map_t = VGG16_net(net_input)
    depth = act_map_t.shape[1]

    # Co-occurrences calcultation
    # Direct cooc_filter
    cooc_filter = ini_cooc_filter(depth, args.cooc_r)
    cooc_map_fix = calc_spatial_cooc(act_map_t, cooc_filter, args.cooc_r)
    cooc_map_fix = cooc_map_fix.cpu().data.squeeze().numpy()

    # Co-occurrences calcultation
    # Learned cooc_filter trained previously
    cooc_filter = np.load(COOC_FILTER_PATH)
    cooc_filter = torch.FloatTensor(cooc_filter).cuda()
    cooc_map_learned = calc_spatial_cooc(act_map_t, cooc_filter, 4)
    cooc_map_learned = cooc_map_learned.cpu().data.squeeze().numpy()

    # Graphic representation
    act_map = act_map_t.cpu().data.squeeze().numpy()
    plot_figures(image, act_map, cooc_map_fix, cooc_map_learned)